In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
database = "/content/drive/MyDrive/CSE508_Winter2023_Dataset"

In [ ]:
pip install beautifulsoup4

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:

from bs4 import BeautifulSoup

In [ ]:
import os
from bs4 import BeautifulSoup

# Change the directory
os.chdir(database)
  
def file_handling(file_path):
  with open(file_path) as fp:
    soup = BeautifulSoup(fp, 'html.parser')

  # storing content in a list format
  content=[]

  # reading all files before editing
  f = open(file_path, "r")
  print(f.read())
  # print(soup.title.text.strip() , soup.find('text').text)

  # appending elemnet to the list 
  content.append(soup.title.text.strip())
  content.append(soup.find('text').text.strip(""))

  # writing in same file the result 
  f = open(file_path, "w")
  for x in content:
    f.write(x)
  f.close()
  content.clear()

  # reading all files after editing
  f = open(file_path, "r")
  print(f.read())


# iterate through all file
for file in os.listdir():
    # Check whether file is in text format or not
        file_path = f"{database}/{file}"
  
        # call read text file function
        file_handling(file_path) 

a simple method of matric structural analysis, part
iv, non-linear problems .
  the method presented in the previous parts is employed to
solve various kinds of nonlinear problems, such as problems concerning
large deflections or buckling, or thermal creep, or inelastic
stress redistribution involving thermal gradients, or design
.  the procedure used in each case is one of direct iteration-i.
e., after one assumes a starting point all subsequent cycles are
self-generating .  simple numerical examples are worked out .



AttributeError: ignored

# **Q1**

In [ ]:
import os
from os import popen
import math
import numpy as np
from collections import Counter
import string
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import nltk
from typing import List
import os
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

# **TF-IDF Matrix**

In [ ]:

def tokenize(doc):
    tokens = word_tokenize(doc.lower())
    tokens = [token for token in tokens if token not in stopwords.words('english') + list(string.punctuation)]
    tokens = [PorterStemmer().stem(token) for token in tokens]
    return tokens


def calculate_tf(tokens, weight_scheme):
    tf = {}
    if weight_scheme == 'binary':
        tf = {token: 1 for token in tokens}
    elif weight_scheme == 'raw':
        tf = Counter(tokens)
    elif weight_scheme == 'term_frequency':
        tf = {token: count/len(tokens) for token, count in Counter(tokens).items()}
    elif weight_scheme == 'log_norm':
        max_count = max(Counter(tokens).values())
        tf = {token: 1 + math.log(count) / math.log(max_count) if max_count > 1 else 0 for token, count in Counter(tokens).items()}
    elif weight_scheme == 'double_norm':
        max_count = max(Counter(tokens).values())
        tf = {token: 0.5 + 0.5 * count / max_count for token, count in Counter(tokens).items()}
    else:
        raise ValueError('Invalid weight_scheme: {}'.format(weight_scheme))
    return tf


def calculate_idf(tokenized_docs, term):
    num_docs_with_term = len([True for doc in tokenized_docs if term in doc])
    return math.log(len(tokenized_docs) / num_docs_with_term)


def calculate_tfidf(tokenized_doc, tf, idf):
    return {term: tf[term] * idf[term] for term in tokenized_doc}


def create_tfidf_matrix(docs, weight_scheme):
    tokenized_docs = [tokenize(doc) for doc in docs]
    vocab = list(set([token for doc in tokenized_docs for token in doc]))
    idf = {term: calculate_idf(tokenized_docs, term) for term in vocab}
    tfidf_matrix = []
    for tokenized_doc in tokenized_docs:
        tf = calculate_tf(tokenized_doc, weight_scheme)
        tfidf = calculate_tfidf(tokenized_doc, tf, idf)
        tfidf_matrix.append(tfidf)
    return tfidf_matrix


docs = []
data_dir = database
for file_name in os.listdir(data_dir):
    file_path = os.path.join(data_dir, file_name)
    with open(file_path, 'r', encoding='utf-8') as f:
        docs.append(f.read())

# Get user query and tokenize it
query = input('Enter your query: ')
query_tokens = tokenize(query)

# Calculate TF-IDF scores for query using all five weight schemes
weight_schemes = ['binary', 'raw', 'term_frequency', 'log_norm', 'double_norm']
for weight_scheme in weight_schemes:
    print('Weight scheme: {}'.format(weight_scheme))
    tfidf_matrix = create_tfidf_matrix(docs, weight_scheme)
    query_tf = calculate_tf(query_tokens, weight_scheme)
    query_tfidf = calculate_tfidf(query_tokens, query_tf, {term: 1 for term in query_tokens})
    for i, doc_tfidf in enumerate(tfidf_matrix):
        doc_score = sum([score for term, score in query_tfidf.items() if term in doc_tfidf])
        print('Doc {}: {}'.format(i+1, doc_score))

# **Jaccard Coefficient**

In [ ]:
def jaccard_coefficient(query: str, document: str) -> float:
    query_tokens = set(query.lower().split())
    document_tokens = set(document.lower().split())
    intersection_size = len(query_tokens.intersection(document_tokens))
    union_size = len(query_tokens.union(document_tokens))
    return intersection_size / union_size

def rank_documents_by_jaccard_coefficient(query: str, documents: List[str], filenames: List[str]) -> List[str]:
    scores = [(filename, jaccard_coefficient(query, document)) for filename, document in zip(filenames, documents)]
    sorted_scores = sorted(scores, key=lambda x: x[1], reverse=True)
    return [filename for filename, _ in sorted_scores][:10]

filenames = os.listdir(database)
documents = [open(os.path.join(database, filename), "r").read() for filename in filenames]

query = input('Enter your query: ')
top_documents = rank_documents_by_jaccard_coefficient(query, documents, filenames)
print(top_documents)

Enter your query: procedure analysis
['cranfield1071', 'cranfield0832', 'cranfield1358', 'cranfield1054', 'cranfield0619', 'cranfield1276', 'cranfield1048', 'cranfield0551', 'cranfield1357', 'cranfield0326']


# **Q2**

In [ ]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer

# Load dataset
df = pd.read_csv('/content/BBC News Sample Solution.csv')

# Drop unnecessary columns
df.drop('ArticleId', axis=1, inplace=True)

# Clean text by removing punctuation and converting to lowercase
df['Text'] = df['Text'].str.replace('[^\w\s]', '').str.lower()

# Tokenize text by splitting it into words
df['Text'] = df['Text'].apply(nltk.word_tokenize)

# Remove stop words
stop_words = set(stopwords.words('english'))
df['Text'] = df['Text'].apply(lambda x: [word for word in x if word not in stop_words])

# Perform stemming or lemmatization
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()
df['Text'] = df['Text'].apply(lambda x: [stemmer.stem(word) for word in x])

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/IPython/core/interactiveshell.py", line 3326, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-45-5d30354c6dbb>", line 8, in <module>
    df = pd.read_csv('/content/BBC News Sample Solution.csv')
  File "/usr/local/lib/python3.9/dist-packages/pandas/util/_decorators.py", line 311, in wrapper
    return func(*args, **kwargs)
  File "/usr/local/lib/python3.9/dist-packages/pandas/io/parsers/readers.py", line 678, in read_csv
    return _read(filepath_or_buffer, kwds)
  File "/usr/local/lib/python3.9/dist-packages/pandas/io/parsers/readers.py", line 575, in _read
    parser = TextFileReader(filepath_or_buffer, **kwds)
  File "/usr/local/lib/python3.9/dist-packages/pandas/io/parsers/readers.py", line 932, in __init__
    self._engine = self._make_engine(f, self.engine)
  File "/usr/local/lib/python3.9/dist-packages/pandas/io/parsers/readers.py", line 1216, in _make_engin

OSError: ignored